In [1]:
import numpy as np
import cv2
import mediapipe as mp
import os

In [3]:
gestures = ['index', 'leftclick', 'palm', 'rock']

In [2]:
os.path.join('index', 'adsf.jpg')

'index\\adsf.jpg'

In [6]:
cap = cv2.VideoCapture(0)

drawing = mp.solutions.drawing_utils
hands = mp.solutions.hands
hand_obj = hands.Hands(max_num_hands = 1)

i = 0

while True:
    i=i+1
    _, frm = cap.read()

    h, w, c = frm.shape

    frm = cv2.flip(frm,1)

    res = hand_obj.process(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))
    foi = frm
    dim = frm.shape
    # Draw the hand landmarks on the image
    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:

            # Draw a rectangle around the hand
            x_max = y_max = 0
            x_min = dim[1]
            y_min= dim[0]
            for landmark in hand_landmarks.landmark:
                x, y = int(landmark.x * frm.shape[1]), int(landmark.y * frm.shape[0])

                if x < x_min:
                    x_min = x
                if x > x_max:
                    x_max = x
                if y < y_min:
                    y_min = y
                if y > y_max:
                    y_max = y
            wid = x_max-x_min
            if wid<200:
                wid_d = (200 - wid)//2

            ht = y_max-y_min
            if ht<250:
                ht_d = (250 - ht)//2
            cv2.rectangle(frm, (x_min-wid_d, y_min-ht_d), (x_max+wid_d, y_max+ht_d), (0, 255, 0), 2)

            foi = frm[y_min-ht_d+2:y_max+ht_d-2, x_min-wid_d+2:x_max+wid_d-2]
            #foi = frm[y_min:y_max, x_min:x_max]
            #drawing.draw_landmarks(frm,res.multi_hand_landmarks[0],hands.HAND_CONNECTIONS) 


    cv2.imshow("Video", foi)

    if cv2.waitKey(1) == ord('s'):
#             i+=1    
            cv2.imwrite(f"C:\\Users\\Rohit\\Music\\Projects\ML\\Gesture-Detection\\data\\rock\\image-{i}.jpg", foi)

    if cv2.waitKey(1) == ord('q'):
        cv2.destroyAllWindows()
        cap.release()
        break

In [18]:
def rename_images(gesture):
    if gesture not in gestures:
        return
    images = os.listdir(os.path.join(gesture))
    for index, img in enumerate(images):
        os.rename(os.path.join('data', gesture, img), os.path.join('data', gesture, f"{gesture}-{index+1}.jpg"))

In [17]:
rename_images('rock')

# Preprocessing and Train Test Splitting the data

In [4]:
import tensorflow as tf

In [21]:
def resize_image(img):
    resized_img = cv2.resize(img, (32, 32))
    return resized_img

In [30]:
# Load the dataset
data = tf.keras.utils.image_dataset_from_directory('data')

# Normalize the pixel values
data = data.map(lambda x, y : (x/255, y))

# Resize the images to (32, 32, 3)
data = data.map(lambda x, y: (tf.image.resize(x, (32, 32)), y))

data_iterator = data.as_numpy_iterator()
train_size = int(len(data)*0.7)
val_size = int(len(data)*0.2)
test_size = int(len(data)*0.1)


Found 444 files belonging to 4 classes.


In [31]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size).skip(val_size).take(test_size)

In [32]:
len(train)

9

In [26]:
train_size

9

In [31]:
type(data)

tensorflow.python.data.ops.dataset_ops.MapDataset

# Building the model

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Input, MaxPooling2D, Dense, Flatten, Dropout

In [67]:
model = Sequential()

In [68]:
model.add(Conv2D(16, (3,3), 1, activation = 'relu', input_shape = (32, 32, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation = 'relu'))
model.add(MaxPooling2D())
# model.add(Conv2D(16, (3,3), 1, activation = 'relu'))
# model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(4, activation = 'Softmax'))

In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 30, 30, 16)        448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 15, 15, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 13, 13, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 6, 6, 32)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 4, 4, 16)          4624      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 2, 2, 16)        

In [69]:
model.compile(optimizer = 'adam', loss = tf.losses.SparseCategoricalCrossentropy(), metrics = ['accuracy'])

# Training the model

In [38]:
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [70]:
hist = model.fit(train, epochs= 25, validation_data=val, callbacks=[tensorboard_callback])

Epoch 1/25
9/9 [==============================] - 1s 46ms/step - loss: 1.3869 - accuracy: 0.2674 - val_loss: 1.3779 - val_accuracy: 0.2188
Epoch 2/25
9/9 [==============================] - 0s 43ms/step - loss: 1.3574 - accuracy: 0.2847 - val_loss: 1.3503 - val_accuracy: 0.3594
Epoch 3/25
9/9 [==============================] - 0s 40ms/step - loss: 1.3116 - accuracy: 0.4479 - val_loss: 1.2601 - val_accuracy: 0.5156
Epoch 4/25
9/9 [==============================] - 0s 41ms/step - loss: 1.2293 - accuracy: 0.5521 - val_loss: 1.1644 - val_accuracy: 0.7500
Epoch 5/25
9/9 [==============================] - 0s 41ms/step - loss: 1.1020 - accuracy: 0.6250 - val_loss: 1.0040 - val_accuracy: 0.7344
Epoch 6/25
9/9 [==============================] - 0s 41ms/step - loss: 0.9389 - accuracy: 0.7326 - val_loss: 0.9204 - val_accuracy: 0.5781
Epoch 7/25
9/9 [==============================] - 0s 42ms/step - loss: 0.7776 - accuracy: 0.7465 - val_loss: 0.7570 - val_accuracy: 0.7656
Epoch 8/25
9/9 [===========

In [40]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [97]:
model.save('model')

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [82]:
import cv2

In [5]:
model = tf.keras.models.load_model('model')

In [6]:
cap = cv2.VideoCapture(0)

drawing = mp.solutions.drawing_utils
hands = mp.solutions.hands
hand_obj = hands.Hands(max_num_hands = 1)

i = 0

while True:
    i=i+1
    _, frm = cap.read()

    h, w, c = frm.shape

    frm = cv2.flip(frm,1)

    res = hand_obj.process(cv2.cvtColor(frm, cv2.COLOR_BGR2RGB))
    foi = frm
    dim = frm.shape
    # Draw the hand landmarks on the image
    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:

            # Draw a rectangle around the hand
            x_max = y_max = 0
            x_min = dim[1]
            y_min= dim[0]
            for landmark in hand_landmarks.landmark:
                x, y = int(landmark.x * frm.shape[1]), int(landmark.y * frm.shape[0])

                if x < x_min:
                    x_min = x
                if x > x_max:
                    x_max = x
                if y < y_min:
                    y_min = y
                if y > y_max:
                    y_max = y
            wid = x_max-x_min
            if wid<200:
                wid_d = (200 - wid)//2

            ht = y_max-y_min
            if ht<250:
                ht_d = (250 - ht)//2
            cv2.rectangle(frm, (x_min-wid_d, y_min-ht_d), (x_max+wid_d, y_max+ht_d), (0, 255, 0), 2)

            foi = frm[y_min-ht_d+2:y_max+ht_d-2, x_min-wid_d+2:x_max+wid_d-2]

#                 cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            #foi = frm[y_min:y_max, x_min:x_max]
            #drawing.draw_landmarks(frm,res.multi_hand_landmarks[0],hands.HAND_CONNECTIONS) 


    resize = tf.image.resize(foi, (32,32))
    yhat = model.predict(np.expand_dims(resize/255, 0))
    cv2.rectangle(foi, (0,0), (640, 40), (245, 117, 16), -1)
    cv2.putText(foi, gestures[np.argmax(yhat[0])] , (3,30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow("Video", foi)
    

#     if cv2.waitKey(1) == ord('s'):
#             i+=1    
#             cv2.imwrite(f"C:\\Users\\Rohit\\Music\\Projects\ML\\Gesture-Detection\\test\\image-{i}.jpg", foi)

    if cv2.waitKey(1) == ord('q'):
        cv2.destroyAllWindows()
        cap.release()
        break

1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 13ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 14ms/step


InvalidArgumentError: {{function_node __wrapped__ResizeBilinear_device_/job:localhost/replica:0/task:0/device:CPU:0}} input image must be of non-zero size [Op:ResizeBilinear]

In [92]:
images = os.listdir(os.path.join('test'))
# print(sorted(images))
for index, img in enumerate(sorted(images)):
    os.rename(os.path.join('test', img), os.path.join('test', f"image-{index+1}.jpg"))

In [76]:
import matplotlib.pyplot as plt

In [44]:
def get_prediction(yhat):
    return gestures[np.argmax(yhat[0])]

In [93]:
for i in range(len(os.listdir('test'))):
    img = cv2.imread(f"test/image-{i+1}.jpg")
    resize = tf.image.resize(img, (32,32))
    yhat = model.predict(np.expand_dims(resize/255, 0))
    print(get_prediction(yhat))
    

1/1 [==============================] - 0s 15ms/step
leftclick
1/1 [==============================] - 0s 15ms/step
palm
1/1 [==============================] - 0s 15ms/step
rock
1/1 [==============================] - 0s 14ms/step
index
1/1 [==============================] - 0s 14ms/step
palm
1/1 [==============================] - 0s 14ms/step
palm
1/1 [==============================] - 0s 15ms/step
palm
1/1 [==============================] - 0s 14ms/step
rock
1/1 [==============================] - 0s 14ms/step
rock
1/1 [==============================] - 0s 15ms/step
index
1/1 [==============================] - 0s 15ms/step
index
1/1 [==============================] - 0s 14ms/step
index
1/1 [==============================] - 0s 15ms/step
leftclick
1/1 [==============================] - 0s 14ms/step
leftclick
1/1 [==============================] - 0s 14ms/step
leftclick
1/1 [==============================] - 0s 15ms/step
leftclick
1/1 [==============================] - 0s 15ms/step
palm
1/